## Overture Maps Data Download Test
### Gavin Rolls

NOTE: Use urbsim kernel to avoid parse error

In [81]:
#Library Imports - using DuckDB for Overture Import
import duckdb
import pandas as pd
import geopandas as gpd

In [3]:
#Config SQL
%pip install ipython-sql duckdb duckdb-engine jupysql --quiet
%pip install --upgrade grpcio --quiet
%load_ext sql

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [5]:
%%sql      
INSTALL httpfs;

LOAD httpfs;

INSTALL spatial;

LOAD spatial;

SET s3_region='us-west-2';

,Success


### Demo Direct Loading of Overture Data (Demo from Data Extraction Docs - Overture Maps)

In [ ]:
#Takes forever - just a proof of concept. It does work, though

# %%sql

# CREATE OR REPLACE VIEW admins_view AS (
#     SELECT
#         *
#     FROM
#         read_parquet('s3://overturemaps-us-west-2/release/2024-05-16-beta.0/theme=admins/type=*/*', filename=true, hive_partitioning=1)
# );

# COPY (
#     SELECT
#            admins.id,
#            admins.subtype,
#            admins.iso_country_code_alpha_2,
#            admins.admin_level,
#            areas.area_id,
#            names.primary AS primary_name,
#            sources[1].dataset AS primary_source,
#            ST_GeomFromWkb(areas.area_geometry) AS geometry
#       FROM admins_view AS admins
#       INNER JOIN (
#            SELECT
#                   id AS area_id,
#                   locality_id,
#                   geometry AS area_geometry
#            FROM admins_view
#        ) AS areas ON areas.locality_id == admins.id
#       WHERE admins.admin_level = 1
#  ) TO 'countries_overture.geojson'
# WITH (FORMAT GDAL, DRIVER 'GeoJSON', SRS 'EPSG:4326');

### Demo Direct Loading of POIs (Mountains)

In [13]:
# %%sql

# COPY(
# SELECT
#    id,
#    names.primary as primary_name,
#    bbox.xmin as x,
#    bbox.ymin as y,
#    ST_GeomFromWKB(geometry) as geometry,
#    categories.main as main_category,
#    sources[1].dataset AS primary_source,
#    confidence
# FROM read_parquet('s3://overturemaps-us-west-2/release/2024-05-16-beta.0/theme=places/type=*/*', filename=true, hive_partitioning=1)
# WHERE main_category = 'mountain' AND confidence > .90
# ORDER BY confidence DESC
# ) TO 'overture_places_mountains_gt90.shp'
# WITH (FORMAT GDAL, DRIVER 'GeoJSON');

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


### Test Building Download - London

In [6]:
#Get London Bounding Box
from geopy.geocoders import Nominatim
from shapely.geometry import box

# Initialize the geolocator
geolocator = Nominatim(user_agent="geoapi")

# Get location data
location = geolocator.geocode("London")

# Get the bounding box
bounding_box = location.raw['boundingbox']

# Convert bounding box to coordinates
min_lat, max_lat = float(bounding_box[0]), float(bounding_box[1])
min_lon, max_lon = float(bounding_box[2]), float(bounding_box[3])

print(min_lon)
print(min_lat)
print(max_lon)
print(max_lat)

-0.5103751
51.2867601
0.3340155
51.6918741


In [48]:
%%sql

LOAD azure;

SET azure_storage_connection_string = 'DefaultEndpointsProtocol=https;AccountName=overturemapswestus2;AccountKey=;EndpointSuffix=core.windows.net';
COPY (
SELECT
    names.primary as primary_name,
    height,
    level,
    ST_GeomFromWKB(geometry) as geometry
FROM read_parquet('azure://release/2024-05-16-beta.0/theme=buildings/type=building/*', filename=true, hive_partitioning=1)
WHERE primary_name IS NOT NULL
AND bbox.xmin > -0.5103751
AND bbox.xmax < 0.3340155
AND bbox.ymin > 51.2867601
AND bbox.ymax < 51.6918741
) TO 'data/london_buildings.geojson'
WITH (FORMAT GDAL, DRIVER 'GeoJSON', SRS 'EPSG:4326');



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


### Test Places Download - London

In [22]:
%%sql

COPY (
    SELECT
        names.primary AS name,
        categories.main as category,
        ROUND(confidence,2) as confidence,
        ST_GeomFromWKB(geometry) as geometry
FROM read_parquet('s3://overturemaps-us-west-2/release/2024-05-16-beta.0/theme=places/*/*')
WHERE
    bbox.xmin BETWEEN -0.5103751 AND 0.3340155 AND
    bbox.ymin BETWEEN 51.2867601 AND 51.6918741
) TO 'data/london_places.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON', SRS 'EPSG:4326');

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [271]:
#Basic overview stats of London Data
buildings = gpd.read_file('data/london_buildings.geojson')
places = gpd.read_file('data/london_places.geojson')

#Count of Features
print("London Building Count: " + str(buildings.shape[0]))
print("London POI count: " + str(places.shape[0]))

#CONT HERE WITH SUMMARY STATS

London Building Count: 61364
London POI count: 343712


### Loading LSOA Employment Data (From BRES)

In [200]:
#Skip the first six rows because they're header information
empl_data = pd.read_csv('data_imports/lsoa_by_industry_london.csv', skiprows=7, delimiter=',')

unnamed_cols = empl_data.columns[empl_data.columns.str.contains('^Unnamed:')]
empl_data.drop(columns=unnamed_cols, inplace=True)

#Separate name into LSOA11CD and LSOA11NM
def split_column(value):
    #Keep Greater London stats
    if value.startswith('gor:'):
        return value, value
    #Split into name and code
    else:
        parts = value.split('lsoa2011:')[1]
        code, name = parts.split(' : ')
        return code.strip(), name.strip()
        return code, name

empl_data[['LSOA11CD', 'LSOA11NM']] = empl_data['Area'].apply(lambda x: pd.Series(split_column(x)))

print("Num Rows Before: " + str(empl_data.shape[0]))

#There appear to be a bunch of duplicates so I'm going to get rid of them now
empl_data.drop_duplicates(inplace=True)

print("Num Rows After: " + str(empl_data.shape[0]))

empl_data.head()

Num Rows Before: 9478
Num Rows After: 4836


,Area,"01 : Crop and animal production, hunting and related service activities",02 : Forestry and logging,03 : Fishing and aquaculture,05 : Mining of coal and lignite,06 : Extraction of crude petroleum and natural gas,07 : Mining of metal ores,08 : Other mining and quarrying,09 : Mining support service activities,10 : Manufacture of food products,...,92 : Gambling and betting activities,93 : Sports activities and amusement and recreation activities,94 : Activities of membership organisations,95 : Repair of computers and personal and household goods,96 : Other personal service activities,97 : Activities of households as employers of domestic personnel,98 : Undifferentiated goods- and services-producing activities of private households for own use,99 : Activities of extraterritorial organisations and bodies,LSOA11CD,LSOA11NM
0,gor:London,1250,1250,400,0,1500,0,450,350,32000,...,15000,56000,57000,17000,62000,0,0,0,gor:London,gor:London
1,lsoa2011:E01000907 : Camden 001A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,E01000907,Camden 001A
2,lsoa2011:E01000908 : Camden 001B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,E01000908,Camden 001B
3,lsoa2011:E01000909 : Camden 001C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,5,0,0,0,E01000909,Camden 001C
4,lsoa2011:E01000912 : Camden 001D,0,0,0,0,0,0,0,0,0,...,0,0,0,0,5,0,0,0,E01000912,Camden 001D


In [313]:
#Get London LSOA Shapefile Data
lsoa_geo = gpd.read_file('data_imports/london_lsoa_shapefiles/LSOA_2011_London_gen_MHW.shp')

#Convert to WGS for consistency
lsoa_geo = lsoa_geo.to_crs(epsg=4326)

lsoa_geo = lsoa_geo.drop(lsoa_geo.columns[list(range(3, 8))], axis = 1)

print("Num Rows: " + str(lsoa_geo.shape[0]))

lsoa_geo.head()

Num Rows: 4835


,LSOA11CD,LSOA11NM,MSOA11CD,USUALRES,HHOLDRES,COMESTRES,POPDEN,HHOLDS,AVHHOLDSZ,geometry
0,E01000001,City of London 001A,E02000001,1465,1465,0,112.9,876,1.7,"POLYGON ((-0.09729 51.52158, -0.09652 51.52027..."
1,E01000002,City of London 001B,E02000001,1436,1436,0,62.9,830,1.7,"POLYGON ((-0.08813 51.51941, -0.08929 51.51752..."
2,E01000003,City of London 001C,E02000001,1346,1250,96,227.7,817,1.5,"POLYGON ((-0.09679 51.52325, -0.09647 51.52282..."
3,E01000005,City of London 001E,E02000001,985,985,0,52.0,467,2.1,"POLYGON ((-0.07323 51.51000, -0.07553 51.50974..."
4,E01000006,Barking and Dagenham 016A,E02000017,1703,1699,4,116.2,543,3.1,"POLYGON ((0.09115 51.53909, 0.09326 51.53787, ..."


### Join Datasets (LSOA Geography + Employment Statistics)

In [315]:
empl_geog = pd.merge(lsoa_geo, empl_data, on = "LSOA11CD")

print("Num Rows: " + str(empl_geog.shape[0]))

empl_geog.head()

Num Rows: 4835


,LSOA11CD,LSOA11NM_x,MSOA11CD,USUALRES,HHOLDRES,COMESTRES,POPDEN,HHOLDS,AVHHOLDSZ,geometry,...,"91 : Libraries, archives, museums and other cultural activities",92 : Gambling and betting activities,93 : Sports activities and amusement and recreation activities,94 : Activities of membership organisations,95 : Repair of computers and personal and household goods,96 : Other personal service activities,97 : Activities of households as employers of domestic personnel,98 : Undifferentiated goods- and services-producing activities of private households for own use,99 : Activities of extraterritorial organisations and bodies,LSOA11NM_y
0,E01000001,City of London 001A,E02000001,1465,1465,0,112.9,876,1.7,"POLYGON ((-0.09729 51.52158, -0.09652 51.52027...",...,250,0,125,400,10,10,0,0,0,City of London 001A
1,E01000002,City of London 001B,E02000001,1436,1436,0,62.9,830,1.7,"POLYGON ((-0.08813 51.51941, -0.08929 51.51752...",...,150,0,100,300,0,35,0,0,0,City of London 001B
2,E01000003,City of London 001C,E02000001,1346,1250,96,227.7,817,1.5,"POLYGON ((-0.09679 51.52325, -0.09647 51.52282...",...,0,0,20,10,0,10,0,0,0,City of London 001C
3,E01000005,City of London 001E,E02000001,985,985,0,52.0,467,2.1,"POLYGON ((-0.07323 51.51000, -0.07553 51.50974...",...,0,20,40,100,150,35,0,0,0,City of London 001E
4,E01000006,Barking and Dagenham 016A,E02000017,1703,1699,4,116.2,543,3.1,"POLYGON ((0.09115 51.53909, 0.09326 51.53787, ...",...,0,0,0,0,0,0,0,0,0,Barking and Dagenham 016A


### Get Building Data at LSOA Level
For convenience, I'll start with LSOA Camden 001A 

In [321]:
#Get Geometry for Camden 001A

lsoa = empl_geog[empl_geog['LSOA11NM_x'] == 'Camden 001A']

lsoa.head()
geom = lsoa['geometry'].iloc[0]
filter_geom = gpd.GeoSeries([geom], crs=buildings.crs)

#Identify all buildings which overlap with LSOA geometry
filtered_buildings = buildings[buildings.geometry.within(filter_geom.unary_union)]

#All places overlapping with LSOA geography
filtered_places = places[places.geometry.within(filter_geom.unary_union)]


print(filtered_buildings)
print(filtered_places)




Empty GeoDataFrame
Columns: [primary_name, height, level, geometry]
Index: []
                                             name  \
230563                               CR7 the GOAT   
230622         Accounting Solutions for Charities   
230624                          The People's Hall   
230626                            Mess Around Ltd   
230627                               3 Point Park   
230628  The Religious Faith of Fish Story Gallery   
230752                         Stoneleigh Terrace   
230756                        58-77 Lulot Gardens   

                                category  confidence  \
230563                              None        0.27   
230622                        accountant        0.68   
230624  landmark_and_historical_building        0.55   
230626                    event_planning        0.48   
230627                              park        0.67   
230628            arts_and_entertainment        0.68   
230752                       real_estate        0.78

### Spatial Indexing?

In [ ]:
# I want to convert all my statistics to an H3 grid but I don't know if this is a scientifically rigorous method just yet